# Targeted customer segment : young people

In [143]:
pip install ortools

Note: you may need to restart the kernel to use updated packages.


In [144]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import matplotlib.colors as mcolors

In [146]:
repo_url = 'https://raw.githubusercontent.com/baertsch/MGT-530-SLO/main/'
vehicle_matrix = pd.read_excel(repo_url + 'data/vhc_matrix_city_excluded.xlsx')
vehicle_matrix = vehicle_matrix.iloc[0:, :1]
vehicle_matrix
v = vehicle_matrix.values.tolist()
v = [item[0] for item in v]
full_data = pd.read_csv(repo_url + 'data/full_data.csv')

In [147]:
npa_coords = full_data[['NPA', 'lat', 'lng']]
depot_row = pd.DataFrame([{'NPA': 0, 'lat': 46.60396069250175, 'lng': 6.538034286509177}])
npa_coords = pd.concat([npa_coords, depot_row], ignore_index=True)
npa_to_coords = npa_coords.set_index('NPA')[['lat', 'lng']].to_dict('index')
def get_color_palette(n):
    palette = sns.color_palette("pastel", n)
    return [mcolors.rgb2hex(color) for color in palette]

def plot_routes_folium(df_results, npa_to_coords):
    depot_coords = npa_to_coords[0]
    m = folium.Map(location=[depot_coords['lat'], depot_coords['lng']], zoom_start=11)
    num_vehicles = df_results[df_results['vehicle_id'] != 'Total'].shape[0]
    colors = get_color_palette(num_vehicles)

    for idx, row in df_results.iterrows():
        if row['vehicle_id'] == 'Total':
            continue
        fg = folium.FeatureGroup(name=f"Vehicle {row['vehicle_id']}")
        route = row['route']
        points = []
        for npa in route:
            try:
                npa_int = int(npa)
            except Exception:
                continue
            coords = npa_to_coords.get(npa_int)
            if coords:
                points.append((coords['lat'], coords['lng']))
        if points:
            folium.PolyLine(
                points,
                color=colors[idx % len(colors)],
                weight=5,
                opacity=1,  # Fully opaque
                popup=f"Vehicle ID: {row['vehicle_id']}"
            ).add_to(fg)
            for lat, lng in points:
                folium.CircleMarker([lat, lng], radius=3, color='black').add_to(fg)
        fg.add_to(m)
    folium.LayerControl(collapsed=False).add_to(m)
    return m
## Scenario 1 Distance Weighted Demand
def create_data_model(subset_distance_matrix, subset_demands, capacities):
    """Stores the data for the problem."""
    data = {}
    # Data multiplied by a factor of 10 to avoid non-integer numbers
    data['distance_matrix'] = subset_distance_matrix
    data['demands'] = subset_demands
    data['vehicle_capacities'] = capacities
    data['num_vehicles'] = len(capacities)
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Collects solution in a DataFrame for easy CSV export, with NPA in route."""
    results = []
    total_distance = 0
    total_load = 0
    unused_vehicles = []

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0
        route = []
        capacity = data['vehicle_capacities'][vehicle_id]

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            demand = data['demands'][node_index]
            route_load += demand
            route.append(node_to_npa[node_index])
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        end_node = manager.IndexToNode(index)
        route.append(node_to_npa[end_node])

        if route_load == 0:
            unused_vehicles.append(vehicle_id)
            continue  # Skip empty routes

        results.append({
            "vehicle_id": vehicle_id,
            "capacity": capacity,
            "route": route,
            "distance_km": route_distance ,
            "load": route_load,
        })

        total_distance += route_distance
        total_load += route_load

    # Add summary row
    results.append({
        "vehicle_id": "Total",
        "capacity": "",
        "route": "",
        "distance_km": total_distance,
        "load": total_load,
    })

    df_results = pd.DataFrame(results)
    print(df_results)
    return df_results

## Let's build the demand

In [148]:
full_data =pd.read_csv(repo_url + 'data/full_data.csv')

In [149]:
# List of columns to sum
ticket_cols = ['wed_tickets', 'thu_tickets', 'fri_tickets', 'sat_tickets', 'sub_tickets', 'all_tickets']

# Group by 'Commune' and sum the ticket columns
aggregated = full_data.groupby("Commune d'annonce référence")[ticket_cols].sum()


In [150]:
# Take the first row for each 'Commune'
first_rows = full_data.drop_duplicates(subset="Commune d'annonce référence",keep='first').copy()
first_rows.drop(columns=ticket_cols, inplace=True)

# Merge the summed ticket columns back onto the first rows
df = first_rows.merge(aggregated, left_on="Commune d'annonce référence",right_index=True)
df.sort_index(inplace=True)
df


,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,N_lv03,lat,lng,distance_to_venoge_km,wed_tickets,thu_tickets,fri_tickets,sat_tickets,sub_tickets,all_tickets
0,Penthaz,1303,Penthaz,5496,VD,2531186.492,1161298.778,fr,1946,19,...,161298.778,46.599406,6.540556,0.74,246.0,268.0,251.0,292.0,59.0,1057.0
1,Penthalaz,1305,Penthalaz,5495,VD,2530510.884,1162919.512,fr,3177,14,...,162919.512,46.613915,6.531495,2.72,237.0,201.0,200.0,191.0,29.0,829.0
2,Bournens,1035,Bournens,5472,VD,2533025.028,1162344.417,fr,532,7,...,162344.417,46.608998,6.564397,3.51,37.0,20.0,17.0,33.0,1.0,107.0
3,Daillens,1306,Daillens,5480,VD,2531837.469,1164336.340,fr,1099,5,...,164336.340,46.626796,6.548602,3.98,74.0,63.0,46.0,70.0,8.0,253.0
4,Sullens,1036,Sullens,5501,VD,2533207.010,1160372.401,fr,1225,12,...,160372.401,46.591278,6.567058,4.12,44.0,60.0,40.0,38.0,4.0,182.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,Lavey-les-Bains,1892,Lavey-Morcles,5406,VD,2567594.394,1116964.820,fr,1014,11,...,116964.820,46.203357,7.018794,68.54,3.0,6.0,2.0,1.0,0.0,12.0
231,Les Moulins,1660,Château-d'Oex,5841,VD,2573258.645,1144459.628,fr,3656,36,...,144459.628,46.450936,7.090599,68.62,9.0,11.0,0.0,0.0,0.0,20.0
233,Cudrefin,1588,Cudrefin,5456,VD,2569165.061,1200556.438,fr,1885,13,...,200556.438,46.955371,7.033553,69.89,23.0,25.0,26.0,8.0,0.0,82.0
235,La Comballaz,1862,Ormont-Dessous,5410,VD,2572060.524,1136176.786,fr,1244,8,...,136176.786,46.376379,7.075504,72.24,0.0,6.0,0.0,2.0,0.0,8.0


In [151]:
distance_matrix = pd.read_csv(repo_url + 'data/distance_matrix.csv',header=None)
distance_matrix 
distance_matrix = distance_matrix[distance_matrix.index.isin(df.index.tolist())]
distance_matrix = distance_matrix.loc[:, distance_matrix.columns.isin(df.index.tolist())]
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,220,221,222,223,226,230,231,233,235,237
0,inf,2.6786,2.8743,4.0302,4.1759,3.8779,4.6759,4.8092,3.5318,4.4526,...,59.8496,59.8536,61.5070,62.5652,64.7759,68.5917,68.6752,69.9441,72.2981,77.0503
1,2.6786,inf,3.9804,3.0074,5.6066,6.3144,4.1600,2.6572,4.4787,5.7512,...,62.0056,62.0096,63.6630,60.6795,66.9319,70.7477,70.8312,68.0584,74.4541,79.2062
2,2.8743,3.9804,inf,4.1825,2.3205,5.8398,7.3631,5.7399,7.6818,2.2389,...,60.0043,60.0083,61.6617,62.4239,64.9306,68.7464,68.8299,69.8029,72.4528,77.2050
3,4.0302,3.0074,4.1825,inf,5.8087,7.6315,6.1501,3.6034,7.4234,5.9533,...,62.2077,62.2117,63.8651,58.5701,67.1340,70.9498,71.0333,65.9490,74.6562,79.4084
4,4.1759,5.6066,2.3205,5.8087,inf,3.2004,8.0162,8.1495,8.3665,2.7008,...,59.8573,59.8613,61.5147,62.2545,64.7836,68.5994,68.6829,69.6334,72.3058,77.0580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,68.5917,70.7477,68.7464,70.9498,68.5994,68.0194,72.4353,72.5938,72.6896,70.1548,...,35.8143,11.3993,55.9854,102.6311,6.7448,inf,52.5613,112.2001,34.1475,20.1333
231,68.6752,70.8312,68.8299,71.0333,68.6829,68.1028,72.5188,72.6773,72.7731,70.2382,...,40.0247,40.0748,7.1694,71.2089,43.3347,52.5613,inf,80.1927,19.4076,54.1762
233,69.9441,68.0584,69.8029,65.9490,69.6334,71.1241,73.7877,69.0061,74.0420,68.2455,...,103.9451,102.8086,73.0245,7.8177,107.9229,112.2001,80.1927,inf,115.7050,120.4571
235,72.2981,74.4541,72.4528,74.6562,72.3058,71.7258,76.1417,76.3002,76.3960,73.8612,...,39.5207,21.6611,23.2902,106.3375,24.9209,34.1475,19.4076,115.7050,inf,35.7624


In [152]:
target = []
for i in range(18,31):
    target += [f'{i} ans']

In [153]:
df['avg_pct_pop_with_ticket_per_day'] = (df['all_tickets']/4)/df["Nb d'habitants"]
df


,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,lat,lng,distance_to_venoge_km,wed_tickets,thu_tickets,fri_tickets,sat_tickets,sub_tickets,all_tickets,avg_pct_pop_with_ticket_per_day
0,Penthaz,1303,Penthaz,5496,VD,2531186.492,1161298.778,fr,1946,19,...,46.599406,6.540556,0.74,246.0,268.0,251.0,292.0,59.0,1057.0,0.135791
1,Penthalaz,1305,Penthalaz,5495,VD,2530510.884,1162919.512,fr,3177,14,...,46.613915,6.531495,2.72,237.0,201.0,200.0,191.0,29.0,829.0,0.065234
2,Bournens,1035,Bournens,5472,VD,2533025.028,1162344.417,fr,532,7,...,46.608998,6.564397,3.51,37.0,20.0,17.0,33.0,1.0,107.0,0.050282
3,Daillens,1306,Daillens,5480,VD,2531837.469,1164336.340,fr,1099,5,...,46.626796,6.548602,3.98,74.0,63.0,46.0,70.0,8.0,253.0,0.057552
4,Sullens,1036,Sullens,5501,VD,2533207.010,1160372.401,fr,1225,12,...,46.591278,6.567058,4.12,44.0,60.0,40.0,38.0,4.0,182.0,0.037143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,Lavey-les-Bains,1892,Lavey-Morcles,5406,VD,2567594.394,1116964.820,fr,1014,11,...,46.203357,7.018794,68.54,3.0,6.0,2.0,1.0,0.0,12.0,0.002959
231,Les Moulins,1660,Château-d'Oex,5841,VD,2573258.645,1144459.628,fr,3656,36,...,46.450936,7.090599,68.62,9.0,11.0,0.0,0.0,0.0,20.0,0.001368
233,Cudrefin,1588,Cudrefin,5456,VD,2569165.061,1200556.438,fr,1885,13,...,46.955371,7.033553,69.89,23.0,25.0,26.0,8.0,0.0,82.0,0.010875
235,La Comballaz,1862,Ormont-Dessous,5410,VD,2572060.524,1136176.786,fr,1244,8,...,46.376379,7.075504,72.24,0.0,6.0,0.0,2.0,0.0,8.0,0.001608


In [154]:
df['demand_from_youth']=round(df[target].sum(axis=1)*0.15).astype(int)
df


,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,lng,distance_to_venoge_km,wed_tickets,thu_tickets,fri_tickets,sat_tickets,sub_tickets,all_tickets,avg_pct_pop_with_ticket_per_day,demand_from_youth
0,Penthaz,1303,Penthaz,5496,VD,2531186.492,1161298.778,fr,1946,19,...,6.540556,0.74,246.0,268.0,251.0,292.0,59.0,1057.0,0.135791,43
1,Penthalaz,1305,Penthalaz,5495,VD,2530510.884,1162919.512,fr,3177,14,...,6.531495,2.72,237.0,201.0,200.0,191.0,29.0,829.0,0.065234,69
2,Bournens,1035,Bournens,5472,VD,2533025.028,1162344.417,fr,532,7,...,6.564397,3.51,37.0,20.0,17.0,33.0,1.0,107.0,0.050282,12
3,Daillens,1306,Daillens,5480,VD,2531837.469,1164336.340,fr,1099,5,...,6.548602,3.98,74.0,63.0,46.0,70.0,8.0,253.0,0.057552,20
4,Sullens,1036,Sullens,5501,VD,2533207.010,1160372.401,fr,1225,12,...,6.567058,4.12,44.0,60.0,40.0,38.0,4.0,182.0,0.037143,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,Lavey-les-Bains,1892,Lavey-Morcles,5406,VD,2567594.394,1116964.820,fr,1014,11,...,7.018794,68.54,3.0,6.0,2.0,1.0,0.0,12.0,0.002959,16
231,Les Moulins,1660,Château-d'Oex,5841,VD,2573258.645,1144459.628,fr,3656,36,...,7.090599,68.62,9.0,11.0,0.0,0.0,0.0,20.0,0.001368,69
233,Cudrefin,1588,Cudrefin,5456,VD,2569165.061,1200556.438,fr,1885,13,...,7.033553,69.89,23.0,25.0,26.0,8.0,0.0,82.0,0.010875,34
235,La Comballaz,1862,Ormont-Dessous,5410,VD,2572060.524,1136176.786,fr,1244,8,...,7.075504,72.24,0.0,6.0,0.0,2.0,0.0,8.0,0.001608,20


In [155]:
a = 0.2
b = 20

# Apply the logistic function to get interest probability
df['P_Y'] = 1 / (1 + np.exp(a * (df['distance_to_venoge_km'] - b)))

# Adjusted demand = original demand × probability
df['demand_from_youth'] = round(df['demand_from_youth'] * df['P_Y']).astype(int)

df

,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,distance_to_venoge_km,wed_tickets,thu_tickets,fri_tickets,sat_tickets,sub_tickets,all_tickets,avg_pct_pop_with_ticket_per_day,demand_from_youth,P_Y
0,Penthaz,1303,Penthaz,5496,VD,2531186.492,1161298.778,fr,1946,19,...,0.74,246.0,268.0,251.0,292.0,59.0,1057.0,0.135791,42,0.979204
1,Penthalaz,1305,Penthalaz,5495,VD,2530510.884,1162919.512,fr,3177,14,...,2.72,237.0,201.0,200.0,191.0,29.0,829.0,0.065234,67,0.969410
2,Bournens,1035,Bournens,5472,VD,2533025.028,1162344.417,fr,532,7,...,3.51,37.0,20.0,17.0,33.0,1.0,107.0,0.050282,12,0.964360
3,Daillens,1306,Daillens,5480,VD,2531837.469,1164336.340,fr,1099,5,...,3.98,74.0,63.0,46.0,70.0,8.0,253.0,0.057552,19,0.960985
4,Sullens,1036,Sullens,5501,VD,2533207.010,1160372.401,fr,1225,12,...,4.12,44.0,60.0,40.0,38.0,4.0,182.0,0.037143,21,0.959921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,Lavey-les-Bains,1892,Lavey-Morcles,5406,VD,2567594.394,1116964.820,fr,1014,11,...,68.54,3.0,6.0,2.0,1.0,0.0,12.0,0.002959,0,0.000061
231,Les Moulins,1660,Château-d'Oex,5841,VD,2573258.645,1144459.628,fr,3656,36,...,68.62,9.0,11.0,0.0,0.0,0.0,20.0,0.001368,0,0.000060
233,Cudrefin,1588,Cudrefin,5456,VD,2569165.061,1200556.438,fr,1885,13,...,69.89,23.0,25.0,26.0,8.0,0.0,82.0,0.010875,0,0.000046
235,La Comballaz,1862,Ormont-Dessous,5410,VD,2572060.524,1136176.786,fr,1244,8,...,72.24,0.0,6.0,0.0,2.0,0.0,8.0,0.001608,0,0.000029


In [156]:
to_exlude = df[df['demand_from_youth'] == 0].index

In [157]:
with_demand = df.drop(to_exlude)
with_demand

,Commune,NPA,Commune d'annonce / District,Commune d'annonce référence,Canton,E,N,Langue,Nb d'habitants,0 an,...,distance_to_venoge_km,wed_tickets,thu_tickets,fri_tickets,sat_tickets,sub_tickets,all_tickets,avg_pct_pop_with_ticket_per_day,demand_from_youth,P_Y
0,Penthaz,1303,Penthaz,5496,VD,2531186.492,1161298.778,fr,1946,19,...,0.74,246.0,268.0,251.0,292.0,59.0,1057.0,0.135791,42,0.979204
1,Penthalaz,1305,Penthalaz,5495,VD,2530510.884,1162919.512,fr,3177,14,...,2.72,237.0,201.0,200.0,191.0,29.0,829.0,0.065234,67,0.969410
2,Bournens,1035,Bournens,5472,VD,2533025.028,1162344.417,fr,532,7,...,3.51,37.0,20.0,17.0,33.0,1.0,107.0,0.050282,12,0.964360
3,Daillens,1306,Daillens,5480,VD,2531837.469,1164336.340,fr,1099,5,...,3.98,74.0,63.0,46.0,70.0,8.0,253.0,0.057552,19,0.960985
4,Sullens,1036,Sullens,5501,VD,2533207.010,1160372.401,fr,1225,12,...,4.12,44.0,60.0,40.0,38.0,4.0,182.0,0.037143,21,0.959921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,L'Auberson,1454,Sainte-Croix,5568,VD,2525526.009,1185222.525,fr,5130,28,...,39.37,20.0,22.0,35.0,26.0,0.0,103.0,0.005019,2,0.020352
173,Prangins,1197,Prangins,5725,VD,2509066.242,1139607.785,fr,4280,42,...,40.06,38.0,30.0,13.0,20.0,0.0,101.0,0.005900,2,0.017775
176,Genolier,1272,Genolier,5718,VD,2506474.376,1143283.641,fr,2012,10,...,40.57,4.0,11.0,8.0,9.0,0.0,32.0,0.003976,1,0.016079
188,St-Légier-La Chiésaz,1806,Blonay - Saint-Légier,5892,VD,2559223.376,1150077.782,fr,12463,61,...,45.42,40.0,40.0,21.0,13.0,0.0,114.0,0.002287,2,0.006157


In [158]:
distance_matrix = distance_matrix[~distance_matrix.index.isin(to_exlude)]
distance_matrix = distance_matrix.loc[:, ~distance_matrix.columns.isin(to_exlude)]
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,153,154,156,163,164,168,173,176,188,199
0,inf,2.6786,2.8743,4.0302,4.1759,3.8779,4.6759,4.8092,3.5318,4.4526,...,36.8370,37.0502,37.2256,38.4266,38.8414,39.4210,40.1119,40.6223,45.4706,49.5759
1,2.6786,inf,3.9804,3.0074,5.6066,6.3144,4.1600,2.6572,4.4787,5.7512,...,38.9930,39.2061,39.3816,40.5826,37.0412,37.5353,42.2679,42.7782,47.6265,47.6902
2,2.8743,3.9804,inf,4.1825,2.3205,5.8398,7.3631,5.7399,7.6818,2.2389,...,36.9917,37.2048,37.3803,38.5813,41.5286,39.2797,40.2666,40.7770,45.6253,49.4347
3,4.0302,3.0074,4.1825,inf,5.8087,7.6315,6.1501,3.6034,7.4234,5.9533,...,39.1951,39.4082,39.5837,40.7847,39.1084,35.4259,42.4700,42.9804,47.8287,45.5809
4,4.1759,5.6066,2.3205,5.8087,inf,3.2004,8.0162,8.1495,8.3665,2.7008,...,36.8447,37.0578,37.2333,38.4343,42.1817,39.1103,40.1196,40.6299,45.4783,49.2653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,39.4210,37.5353,39.2797,35.4259,39.1103,40.6010,43.2646,38.4830,43.5189,37.7224,...,72.2067,72.3462,72.4727,73.6474,48.4511,inf,75.2153,75.7257,80.5740,51.7038
173,40.1119,42.2679,40.2666,42.4700,40.1196,36.9149,43.9555,44.1140,34.4871,41.6750,...,65.6825,65.8220,4.3664,24.3427,33.2717,75.2153,inf,6.7730,73.5845,85.6364
176,40.6223,42.7782,40.7770,42.9804,40.6299,37.4253,44.4659,44.6243,34.9975,42.1853,...,66.1929,66.3323,3.8212,17.5188,29.5444,75.7257,6.7730,inf,73.9546,86.0065
188,45.4706,47.6265,45.6253,47.8287,45.4783,44.8982,49.3142,49.4727,49.5685,47.0336,...,8.4738,10.2956,70.5758,71.7505,83.7059,80.5740,73.5845,73.9546,inf,46.6743


In [159]:
demand = with_demand[['Commune','NPA',"Commune d'annonce / District",'demand_from_youth','distance_to_venoge_km']]

In [160]:
demand['demand_from_youth']= demand['demand_from_youth'].astype(int)

/var/folders/3d/zlkn8h8d2yx5345bf7csn9bh0000gn/T/ipykernel_34415/548918776.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demand['demand_from_youth']= demand['demand_from_youth'].astype(int)


In [161]:
demand['demand_from_youth'].sum(axis=0)

7541

In [162]:
demand[demand['demand_from_youth']>100]

,Commune,NPA,Commune d'annonce / District,demand_from_youth,distance_to_venoge_km
6,Cossonay-Ville,1304,Cossonay,126,4.72
11,Cheseaux-sur-Lausanne,1033,Cheseaux-sur-Lausanne,108,7.81
14,Crissier,1023,Crissier,340,8.59
17,Romanel-sur-Lausanne,1032,Lausanne,4003,9.17
23,Ecublens VD,1024,Ecublens (VD),454,10.60
28,Chavannes-près-Renens,1022,Chavannes-près-Renens,420,11.43
36,St-Sulpice VD,1025,Saint-Sulpice (VD),144,14.13
37,Le Mont-sur-Lausanne,1052,Le Mont-sur-Lausanne,155,14.30
45,Epalinges,1066,Epalinges,179,16.67
53,Les Monts-de-Pully,1068,Pully,235,17.96


In [139]:
import os

output_dir = "data"
os.makedirs(output_dir, exist_ok=True)


In [ ]:
demand.to_csv(os.path.join(output_dir,'demand_from_youth.csv'),index=False)
distance_matrix.to_csv(os.path.join(output_dir,'distance_matrix_youth.csv'),index=False)

In [212]:
demand1 = pd.read_csv(repo_url + 'data/Split_Demand.csv')

In [163]:
demand

,Commune,NPA,Commune d'annonce / District,demand_from_youth,distance_to_venoge_km
0,Penthaz,1303,Penthaz,42,0.74
1,Penthalaz,1305,Penthalaz,67,2.72
2,Bournens,1035,Bournens,12,3.51
3,Daillens,1306,Daillens,19,3.98
4,Sullens,1036,Sullens,21,4.12
...,...,...,...,...,...
168,L'Auberson,1454,Sainte-Croix,2,39.37
173,Prangins,1197,Prangins,2,40.06
176,Genolier,1272,Genolier,1,40.57
188,St-Légier-La Chiésaz,1806,Blonay - Saint-Légier,2,45.42


In [169]:
demand['demand_from_youth'].sum(axis=0)

7541

In [190]:
demand1 = demand1.merge(full_data[['NPA','distance_to_venoge_km']],on='NPA',how='left')
demand1

,Commune,NPA,Commune d'annonce / District,demand_from_youth,distance_to_venoge_km_x,distance_to_venoge_km_y
0,Penthaz,1303,Penthaz,42,0.74,0.74
1,Penthalaz,1305,Penthalaz,67,2.72,2.72
2,Bournens,1035,Bournens,12,3.51,3.51
3,Daillens,1306,Daillens,19,3.98,3.98
4,Sullens,1036,Sullens,21,4.12,4.12
...,...,...,...,...,...,...
172,L'Auberson,1454,Sainte-Croix,2,39.37,39.37
173,Prangins,1197,Prangins,2,40.06,40.06
174,Genolier,1272,Genolier,1,40.57,40.57
175,St-Légier-La Chiésaz,1806,Blonay - Saint-Légier,2,45.42,45.42


In [213]:
demand

,Commune,NPA,Commune d'annonce / District,demand_from_youth,distance_to_venoge_km
0,Penthaz,1303,Penthaz,42,0.74
1,Penthalaz,1305,Penthalaz,67,2.72
2,Bournens,1035,Bournens,12,3.51
3,Daillens,1306,Daillens,19,3.98
4,Sullens,1036,Sullens,21,4.12
...,...,...,...,...,...
168,L'Auberson,1454,Sainte-Croix,2,39.37
173,Prangins,1197,Prangins,2,40.06
176,Genolier,1272,Genolier,1,40.57
188,St-Légier-La Chiésaz,1806,Blonay - Saint-Légier,2,45.42


In [226]:
demand

,Commune,NPA,Commune d'annonce / District,demand_from_youth,distance_to_venoge_km
0,Penthaz,1303,Penthaz,42,0.74
1,Penthalaz,1305,Penthalaz,67,2.72
2,Bournens,1035,Bournens,12,3.51
3,Daillens,1306,Daillens,19,3.98
4,Sullens,1036,Sullens,21,4.12
...,...,...,...,...,...
168,L'Auberson,1454,Sainte-Croix,2,39.37
173,Prangins,1197,Prangins,2,40.06
176,Genolier,1272,Genolier,1,40.57
188,St-Légier-La Chiésaz,1806,Blonay - Saint-Légier,2,45.42


In [229]:
demand[demand['NPA']==1803]

,Commune,NPA,Commune d'annonce / District,demand_from_youth,distance_to_venoge_km
120,Chardonne,1803,Chardonne,6,31.29


In [231]:
distance_matrix.iloc[119]

IndexError: single positional indexer is out-of-bounds

In [235]:
demand1.reset_index(drop=True,inplace=True)
demand1

,Commune,NPA,Commune d'annonce / District,demand_from_youth,distance_to_venoge_km
0,Penthaz,1303,Penthaz,42,0.74
1,Penthalaz,1305,Penthalaz,67,2.72
2,Bournens,1035,Bournens,12,3.51
3,Daillens,1306,Daillens,19,3.98
4,Sullens,1036,Sullens,21,4.12
...,...,...,...,...,...
172,L'Auberson,1454,Sainte-Croix,2,39.37
173,Prangins,1197,Prangins,2,40.06
174,Genolier,1272,Genolier,1,40.57
175,St-Légier-La Chiésaz,1806,Blonay - Saint-Légier,2,45.42


In [228]:
demand1.iloc[153:155,:]

,Commune,NPA,Commune d'annonce / District,demand_from_youth,distance_to_venoge_km
153,Corseaux,1802,Corseaux,4,30.96
154,Chardonne,1803,Chardonne,6,31.29


In [237]:
demand1['NPA'] = demand1['NPA'].astype(str).str.strip()
demand['NPA'] = demand['NPA'].astype(str).str.strip()

/var/folders/3d/zlkn8h8d2yx5345bf7csn9bh0000gn/T/ipykernel_34415/2340667725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demand['NPA'] = demand['NPA'].astype(str).str.strip()


In [242]:
demand1

,Commune,NPA,Commune d'annonce / District,demand_from_youth,distance_to_venoge_km
0,Penthaz,1303,Penthaz,42,0.74
1,Penthalaz,1305,Penthalaz,67,2.72
2,Bournens,1035,Bournens,12,3.51
3,Daillens,1306,Daillens,19,3.98
4,Sullens,1036,Sullens,21,4.12
...,...,...,...,...,...
172,L'Auberson,1454,Sainte-Croix,2,39.37
173,Prangins,1197,Prangins,2,40.06
174,Genolier,1272,Genolier,1,40.57
175,St-Légier-La Chiésaz,1806,Blonay - Saint-Légier,2,45.42


In [251]:
new_df = []

for _, row in demand1.iterrows():
    npa = row['NPA']
    print(_)
    print(npa)
    if npa in demand['NPA'].values:
        index = demand[demand['NPA'] == npa].index.values[0]
        print(index,'\n')
        distance_row = distance_matrix.iloc[index:index+1, :]
        new_row = distance_row.values[0]
        print(new_row)
        new_df.append(new_row)
    else:
        print(f"NPA {npa} not found in demand")

new_df

0
1303
0 

[    inf  2.6786  2.8743  4.0302  4.1759  3.8779  4.6759  4.8092  3.5318
  4.4526  5.8333  7.8646  8.012   8.5356  6.6482  8.8323  9.1189  9.221
  8.7858  9.3326  9.2563  9.7679 10.3962 10.6561 10.306  10.841  11.3557
 11.4835 12.6105 12.5086 12.6886 13.2747 14.1837 14.354  15.7995 16.7249
 17.3251 17.3536 17.3824 17.9425 18.0157 18.4821 19.5158 19.5809 19.8786
 20.0702 20.1988 20.2004 20.6106 21.0312 21.3205 21.5007 21.8243 22.2954
 21.6146 21.9079 22.9513 23.1938 22.9296 23.8183 23.8325 23.9884 24.2716
 24.6209 24.8784 24.8851 25.168  25.2321 25.2749 25.1803 25.8704 26.3325
 26.3864 26.4556 26.4591 26.5673 26.6923 26.7876 26.2432 27.2994 27.3268
 27.9944 28.9431 29.656  29.8253 29.927  30.0699 30.2266 30.2546 30.3253
 30.4046 30.6263 30.2372 30.2631 30.9759 31.0139 31.3479 32.0125 32.0967
 32.3841 32.5254 32.0435 33.3832 33.391  32.8669 33.7072 33.984  34.4754
 35.115  36.837  37.0502 37.2256 38.4266 38.8414 39.421  40.1119 40.6223
 45.4706 49.5759]
1
1305
1 

[ 2.6786    

IndexError: index 0 is out of bounds for axis 0 with size 0

In [233]:
demand1

,Commune,NPA,Commune d'annonce / District,demand_from_youth,distance_to_venoge_km
0,Penthaz,1303,Penthaz,42,0.74
1,Penthalaz,1305,Penthalaz,67,2.72
2,Bournens,1035,Bournens,12,3.51
3,Daillens,1306,Daillens,19,3.98
4,Sullens,1036,Sullens,21,4.12
...,...,...,...,...,...
172,L'Auberson,1454,Sainte-Croix,2,39.37
173,Prangins,1197,Prangins,2,40.06
174,Genolier,1272,Genolier,1,40.57
175,St-Légier-La Chiésaz,1806,Blonay - Saint-Légier,2,45.42


In [210]:
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,153,154,156,163,164,168,173,176,188,199
0,inf,2.6786,2.8743,4.0302,4.1759,3.8779,4.6759,4.8092,3.5318,4.4526,...,36.8370,37.0502,37.2256,38.4266,38.8414,39.4210,40.1119,40.6223,45.4706,49.5759
1,2.6786,inf,3.9804,3.0074,5.6066,6.3144,4.1600,2.6572,4.4787,5.7512,...,38.9930,39.2061,39.3816,40.5826,37.0412,37.5353,42.2679,42.7782,47.6265,47.6902
2,2.8743,3.9804,inf,4.1825,2.3205,5.8398,7.3631,5.7399,7.6818,2.2389,...,36.9917,37.2048,37.3803,38.5813,41.5286,39.2797,40.2666,40.7770,45.6253,49.4347
3,4.0302,3.0074,4.1825,inf,5.8087,7.6315,6.1501,3.6034,7.4234,5.9533,...,39.1951,39.4082,39.5837,40.7847,39.1084,35.4259,42.4700,42.9804,47.8287,45.5809
4,4.1759,5.6066,2.3205,5.8087,inf,3.2004,8.0162,8.1495,8.3665,2.7008,...,36.8447,37.0578,37.2333,38.4343,42.1817,39.1103,40.1196,40.6299,45.4783,49.2653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,39.4210,37.5353,39.2797,35.4259,39.1103,40.6010,43.2646,38.4830,43.5189,37.7224,...,72.2067,72.3462,72.4727,73.6474,48.4511,inf,75.2153,75.7257,80.5740,51.7038
173,40.1119,42.2679,40.2666,42.4700,40.1196,36.9149,43.9555,44.1140,34.4871,41.6750,...,65.6825,65.8220,4.3664,24.3427,33.2717,75.2153,inf,6.7730,73.5845,85.6364
176,40.6223,42.7782,40.7770,42.9804,40.6299,37.4253,44.4659,44.6243,34.9975,42.1853,...,66.1929,66.3323,3.8212,17.5188,29.5444,75.7257,6.7730,inf,73.9546,86.0065
188,45.4706,47.6265,45.6253,47.8287,45.4783,44.8982,49.3142,49.4727,49.5685,47.0336,...,8.4738,10.2956,70.5758,71.7505,83.7059,80.5740,73.5845,73.9546,inf,46.6743


In [203]:
distance_matrix1.reset_index(drop=True, inplace=True)
distance_matrix1 

,0,1,2,3,4,5,6,7,8,9,...,167,168,169,170,171,172,173,174,175,176
0,0.0000,3.0000,5.0000,6.0000,8.0000,11.0000,13.0000,13.0000,14.0000,16.0000,...,191.0000,204.0000,209.0000,212.0000,225.0000,240.0000,1.0000,2.0000,9.0000,18.0000
1,inf,2.6786,2.6786,4.0302,4.0302,4.1759,4.1759,4.1759,4.8092,4.8092,...,39.3266,42.2405,44.1859,45.4706,50.5653,56.8572,inf,inf,4.1759,4.8092
2,2.6786,inf,inf,3.0074,3.0074,5.6066,5.6066,5.6066,2.6572,2.6572,...,37.4409,44.3965,46.3419,47.6265,52.7213,59.0131,2.6786,2.6786,5.6066,2.6572
3,2.6786,inf,inf,3.0074,3.0074,5.6066,5.6066,5.6066,2.6572,2.6572,...,37.4409,44.3965,46.3419,47.6265,52.7213,59.0131,2.6786,2.6786,5.6066,2.6572
4,4.0302,3.0074,3.0074,inf,inf,5.8087,5.8087,5.8087,3.6034,3.6034,...,35.3315,44.5986,46.5440,47.8287,52.9234,59.2153,4.0302,4.0302,5.8087,3.6034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,50.5653,52.7213,52.7213,52.9234,52.9234,50.5730,50.5730,50.5730,54.5674,54.5674,...,85.1978,13.0623,7.0120,83.6516,inf,95.0799,50.5653,50.5653,50.5730,54.5674
173,56.8572,59.0131,59.0131,59.2153,59.2153,56.8648,56.8648,56.8648,60.8592,60.8592,...,91.4896,86.9595,88.8190,28.7767,95.0799,inf,56.8572,56.8572,56.8648,60.8592
174,inf,2.6786,2.6786,4.0302,4.0302,4.1759,4.1759,4.1759,4.8092,4.8092,...,39.3266,42.2405,44.1859,45.4706,50.5653,56.8572,inf,inf,4.1759,4.8092
175,inf,2.6786,2.6786,4.0302,4.0302,4.1759,4.1759,4.1759,4.8092,4.8092,...,39.3266,42.2405,44.1859,45.4706,50.5653,56.8572,inf,inf,4.1759,4.8092


In [191]:
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,153,154,156,163,164,168,173,176,188,199
0,inf,2.6786,2.8743,4.0302,4.1759,3.8779,4.6759,4.8092,3.5318,4.4526,...,36.8370,37.0502,37.2256,38.4266,38.8414,39.4210,40.1119,40.6223,45.4706,49.5759
1,2.6786,inf,3.9804,3.0074,5.6066,6.3144,4.1600,2.6572,4.4787,5.7512,...,38.9930,39.2061,39.3816,40.5826,37.0412,37.5353,42.2679,42.7782,47.6265,47.6902
2,2.8743,3.9804,inf,4.1825,2.3205,5.8398,7.3631,5.7399,7.6818,2.2389,...,36.9917,37.2048,37.3803,38.5813,41.5286,39.2797,40.2666,40.7770,45.6253,49.4347
3,4.0302,3.0074,4.1825,inf,5.8087,7.6315,6.1501,3.6034,7.4234,5.9533,...,39.1951,39.4082,39.5837,40.7847,39.1084,35.4259,42.4700,42.9804,47.8287,45.5809
4,4.1759,5.6066,2.3205,5.8087,inf,3.2004,8.0162,8.1495,8.3665,2.7008,...,36.8447,37.0578,37.2333,38.4343,42.1817,39.1103,40.1196,40.6299,45.4783,49.2653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,39.4210,37.5353,39.2797,35.4259,39.1103,40.6010,43.2646,38.4830,43.5189,37.7224,...,72.2067,72.3462,72.4727,73.6474,48.4511,inf,75.2153,75.7257,80.5740,51.7038
173,40.1119,42.2679,40.2666,42.4700,40.1196,36.9149,43.9555,44.1140,34.4871,41.6750,...,65.6825,65.8220,4.3664,24.3427,33.2717,75.2153,inf,6.7730,73.5845,85.6364
176,40.6223,42.7782,40.7770,42.9804,40.6299,37.4253,44.4659,44.6243,34.9975,42.1853,...,66.1929,66.3323,3.8212,17.5188,29.5444,75.7257,6.7730,inf,73.9546,86.0065
188,45.4706,47.6265,45.6253,47.8287,45.4783,44.8982,49.3142,49.4727,49.5685,47.0336,...,8.4738,10.2956,70.5758,71.7505,83.7059,80.5740,73.5845,73.9546,inf,46.6743


In [176]:
pd.read_csv(repo_url + 'data/adjusted_dm_wed.csv',header=None)

,0,1,2,3,4,5,6,7,8,9,...,227,228,229,230,231,232,233,234,235,236
0,NaN,0.0000,0.0000,0.0000,1.0000,1.0000,1.0000,2.0000,3.0000,4.0000,...,142.0000,146.0000,148.0000,149.0000,154.0000,160.0000,161.0000,163.0000,165.0000,173.0000
1,0.0,inf,inf,inf,2.6786,2.6786,2.6786,2.8743,4.0302,4.1759,...,34.4754,35.1150,35.7612,35.7723,37.0502,37.8368,38.1101,38.4266,38.9650,40.1119
2,0.0,inf,inf,inf,2.6786,2.6786,2.6786,2.8743,4.0302,4.1759,...,34.4754,35.1150,35.7612,35.7723,37.0502,37.8368,38.1101,38.4266,38.9650,40.1119
3,0.0,inf,inf,inf,2.6786,2.6786,2.6786,2.8743,4.0302,4.1759,...,34.4754,35.1150,35.7612,35.7723,37.0502,37.8368,38.1101,38.4266,38.9650,40.1119
4,1.0,2.6786,2.6786,2.6786,inf,inf,inf,3.9804,3.0074,5.6066,...,32.5897,37.2709,33.8755,33.8866,39.2061,39.9928,40.2660,40.5826,37.1648,42.2679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,160.0,37.8368,37.8368,37.8368,39.9928,39.9928,39.9928,37.9915,40.1949,37.8445,...,67.6181,60.8330,68.9039,81.8360,2.8239,inf,2.3896,64.9919,83.3549,66.6772
233,161.0,38.1101,38.1101,38.1101,40.2660,40.2660,40.2660,38.2647,40.4681,38.1177,...,67.8913,61.1063,69.1771,82.1093,4.0103,2.3896,inf,65.2703,83.6333,66.9556
234,163.0,38.4266,38.4266,38.4266,40.5826,40.5826,40.5826,38.5813,40.7847,38.4343,...,68.2078,13.8851,69.4937,82.4258,64.1367,64.9919,65.2703,inf,24.1395,24.3427
235,165.0,38.9650,38.9650,38.9650,37.1648,37.1648,37.1648,41.6522,39.2320,42.3053,...,55.0692,32.2362,56.3550,69.2871,82.4996,83.3549,83.6333,24.1395,inf,39.5889


In [ ]:
npa_list = demand_wed_df.iloc[1:, 2:3].reset_index(drop=True).values.tolist()
npa_list = [int(float(item[0])) for item in npa_list]
node_to_npa = [0] + npa_list  # index 0 is depot

def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_wed, subset_demands=d_wed, capacities=v_wed_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        df_results= print_solution(data, manager, routing, solution)
        m = plot_routes_folium(df_results, npa_to_coords)
        display(m)

    return df_results

df_results_wed2 = main()